In [2]:
import numpy as np
import torch

Softthresholding function

In [17]:
def softthreshold(x, shrinkage):

    # batch_size, height, width = x.shape[0], x.shape[1], x.shape[2]
    
    # compare each pixels in x with shrinkage value
    x_thx = 0

    if x > shrinkage:
        x_thx = x - shrinkage
    elif x < -shrinkage:
        x_thx = x + shrinkage
    else:
        x_thx = 0
    
    return x_thx

Test the soft thresholding function

In [18]:
x_in = np.arange(-10, 11, 1)
print(x_in)

# apply soft thresholding
for x in x_in:
    x_out = softthreshold(x, 0.1)
    print(x_out)

[-10  -9  -8  -7  -6  -5  -4  -3  -2  -1   0   1   2   3   4   5   6   7
   8   9  10]
0
0
0
0
0
0
0
0
0
0
0
0.9
1.9
2.9
3.9
4.9
5.9
6.9
7.9
8.9
9.9
